In [45]:
import os
import json
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
import gspread
# from gspread_dataframe import set_with_dataframe
# from io import BytesIO
# import openpyxl
import re
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Get environment variables
SERVICE_ACCOUNT_FILE = os.getenv("SERVICE_ACCOUNT_FILE")
SCOPES = os.getenv("API_SCOPES").split(",")

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=SCOPES
)

# Setup Drive and Sheets API clients
drive_service = build('drive', 'v3', credentials=credentials)
gc = gspread.authorize(credentials)

In [46]:
def list_google_sheets_in_first_level_subfolders(drive_service, root_folder_id):
    sheets = []

    # Step 1: Get first-level sub-folders
    subfolders_query = f"'{root_folder_id}' in parents and mimeType='application/vnd.google-apps.folder'"
    subfolders = drive_service.files().list(
        q=subfolders_query,
        spaces='drive',
        fields='files(id, name)'
    ).execute().get("files", [])

    print(f"📁 Found {len(subfolders)} subfolders")

    # Step 2: In each subfolder, get Google Sheets
    for folder in subfolders:
        folder_id = folder['id']
        folder_name = folder['name']

        sheet_query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet'"
        sheet_files = drive_service.files().list(
            q=sheet_query,
            spaces='drive',
            fields='files(id, name, createdTime)'
        ).execute().get("files", [])

        for sheet in sheet_files:
            print(f"{sheet}")
            sheets.append({
                "id": sheet["id"],
                "name": sheet["name"],
                "createdTime": sheet["createdTime"],
                "folder": folder_name,
                "folder_id": folder_id
            })

    return sheets

In [47]:
# Load environment variables from the .env file
load_dotenv()

DRIVER_PARTY_ROOT_FOLDER_ID = os.getenv("DRIVER_PARTY_ROOT_FOLDER_ID")

sheets = list_google_sheets_in_first_level_subfolders(
    drive_service, DRIVER_PARTY_ROOT_FOLDER_ID
)

print(f"✅ Found {len(sheets)} Google Sheets:")

dates = []
party_files_counter = 0
for s in sheets:
    print(f"{s}")
    name_arrayed = s["name"].split(" ")
    for _ in range(len(name_arrayed)):
        earlyRegistration = re.search("תגובות", s["name"])
        if earlyRegistration != None:
            party_files_counter += 1
            # print(f"- {s['name']} (Folder: {s['folder']}) → ID: {s['id']}")
            dates.append(
                {
                    "file_name": s["name"],
                    "folder_name": s["folder"],
                    "id": s["id"],
                    "created_time": s["createdTime"],
                }
            )
            # Found Google sheet in folder, move to next folder
            break


# print(f"✅ Found {party_files_counter} Google Sheets with 'תגובות' in their names:")
# print(dates)

📁 Found 30 subfolders
{'id': '1Vh3HgS-c0VYczLnwY3rVgAUDCYqSbtUeWiLdZEN2eo8', 'name': 'עותק של הרשמה מוקדמת למסיבת הליין OhSnap (תגובות)', 'createdTime': '2022-06-07T14:39:38.690Z'}
{'id': '12M-reVIlyQEctWmiFrITpky_Q5UOL7K7czTQWyxYgGk', 'name': 'לוז קופה 23.6', 'createdTime': '2022-06-19T16:59:21.870Z'}
{'id': '1G_kmdFNmcB3M3FXzObnqTsBOfQCwzW6DO4zJPRmvtes', 'name': 'טופס הרשמה מראש אוסנאפ  (תגובות)', 'createdTime': '2025-04-25T09:16:39.253Z'}
{'id': '1ItG3yNGbRo6BydqmhlA0yUIR3ebtPwICczBebs2Nu78', 'name': 'קופות אוסנאפ', 'createdTime': '2025-01-20T09:24:42.481Z'}
{'id': '1zmJoSNg-LNRH6mxm5U4J7o1_JtL88CfUgISAHMzLG7Y', 'name': 'טופס הרשמה מראש אוסנאפ  (תגובות)', 'createdTime': '2025-02-27T17:22:36.206Z'}
{'id': '13L6lLUA0dEVw9mrXjpa-8gIs1Qm6WN7iFN_k1a19kPg', 'name': ' טופס הרשמה מראש אוסנאפ  (תגובות)', 'createdTime': '2024-12-29T14:51:19.875Z'}
{'id': '1lGz3v2rTzNndT2HgydfqDirOzN1I7p5lgaPQzFzAWss', 'name': 'טופס הרשמה מראש אוסנאפ (תגובות)', 'createdTime': '2024-11-15T09:43:20.889Z'}
{'id':

In [56]:
# Load environment variables from the .env file
load_dotenv()

CACHED_FILE_NAME = os.getenv("CACHED_FILE_NAME")
# Initialize an empty set to store unique column names
columns_set = set()

# Define the cache file
CACHE_FILE = os.path.join(os.getcwd(), CACHED_FILE_NAME)

# Load the cache if it exists
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r") as f:
        cache = json.load(f)
else:
    cache = {}

# Iterate through all files in the dates object
for sheet_info in dates:
    file_id = sheet_info["id"]
    folder_name = sheet_info["folder_name"]

    # Check if the file is already cached
    if (
        folder_name in cache
        and "files" in cache[folder_name]
        and file_id in cache[folder_name]["files"]
    ):
        print(f"Loading data for file")  # '{sheet_info['folder_name']}' from cache.")
        data = cache[folder_name]["files"][file_id]
    else:
        print(
            f"Fetching data for file"
        )  # '{sheet_info['folder_name']}' from Google Sheets.")

        # Open the Google Sheet by ID
        sheet = gc.open_by_key(file_id)

        # Access the first worksheet
        worksheet = sheet.get_worksheet(0)  # 0 is the index of the first worksheet

        # Fetch all rows at once (including headers)
        all_rows = worksheet.get_all_values()  # Single API call to fetch all data

        # Extract headers from the first row
        headers = all_rows[0]  # First row is assumed to be the header row
        # print(f"Headers in sheet '{sheet_info['folder_name']}': {headers}")

        # Extract data from the remaining rows
        data = [dict(zip(headers, row)) for row in all_rows[1:]]  # Map rows to headers

        # Cache the data
        created_time = sheet_info["created_time"]  # Ensure created_time is defined
        print(f"Created time type '{type(sheet_info['created_time'])}': {created_time}")
        if folder_name in cache and "files" in cache[folder_name]:
            cache[folder_name]["files"].update({file_id: data})
        else:
            cache[folder_name] = {
                "created_time": created_time,
                "files": {file_id: data},
            }

        # Iterate through the data list
        for row in data:
            # Add all keys (column names) from the current dictionary to the set
            columns_set.update(row.keys())

        # Save the updated cache to the file
        with open(CACHE_FILE, "w") as f:
            json.dump(cache, f, ensure_ascii=False, indent=4)

# Print the unique column names
# print("Unique columns across all sheets:", columns_set)

Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file
Loading data for file


In [16]:
# Load the cached sheets data
with open(CACHE_FILE, "r") as f:
    cached_sheets = json.load(f)

df_list = []

# Iterate through each file ID and its corresponding data
for folder_name, files in cached_sheets.items():
    for file_id, rows in files.items():
        # Flatten the rows for the current file ID
        df = pd.json_normalize(rows)
        df["file_id"] = file_id  # Add a column for the file ID
        df["folder_name"] = folder_name  # Add a column for the file ID
        df_list.append(df)

# Concatenate all DataFrames into one
final_df = pd.concat(df_list, ignore_index=True)

# Remove columns with empty or whitespace-only headers
final_df = final_df.loc[:, ~(final_df.columns.str.strip() == "")]

# Remove columns where all values are either NaN or empty strings
final_df = final_df.loc[:, ~(final_df.isna() | (final_df == '')).all(axis=0)]

def merge_and_rename(content, new_column_name, df):
    # Identify columns that contain the word "שם"
    columns_containing = [col for col in df.columns if content in col.lower()]
    # Merge the columns into a single column named "full_name"
    df[new_column_name] = df[columns_containing].fillna("").agg(" ".join, axis=1)
    df = df.drop(columns=columns_containing)
    return df


## NAME
final_df = merge_and_rename("שם", "full_name", final_df)

## SPECIAL REQUESTS
final_df = merge_and_rename("הערות", "special_requests", final_df)

## SONGS REQUESTS
final_df = merge_and_rename("song", "song_requests", final_df)

# Rename columns to English
final_df = final_df.rename(columns={
    "חותמת זמן": "timestamp",
    "הגיע.ה?": "arrived",
})


##ARRIVED
# Step 1: Make sure empty fields are treated correctly
final_df['arrived'] = final_df['arrived'].replace('', pd.NA)

# Step 2: Update 'arrived_marker' directly
final_df.loc[(final_df['arrived'].notna()) & (final_df['arrived_marker'] == '#ffffff'), 'arrived'] = True

# Step 3: Update 'arrived' True for colored cells
final_df.loc[final_df['arrived_marker'] != "#ffffff", 'arrived'] = True

# Step 4: Update 'arrived' False if has white background
final_df.loc[(final_df['arrived_marker'] == '#ffffff'), 'arrived'] = False

##FULL_NAME
# Step 1: Remove leading and trailing whitespace
final_df['full_name'] = final_df['full_name'].str.strip()

#Step 2: Filter out rows with empty 'full_name'
final_df = final_df[final_df['full_name'].str.strip() != '']

In [17]:
## TOP 10 REGISTRATIONS
top_registrations = (
    final_df.groupby("full_name")
    .size()  # faster and lighter than count() here
    .sort_values(ascending=False)
    .head(10)
)

# pd.DataFrame(top_registrations, index=None).rename(columns={0: "Top Registrations"})

In [18]:
## TOP 10 ARRIVALS & REGISTRATIONS
top_arrivals = (
    final_df[final_df['arrived'] == True]
    .groupby('full_name')
    .size()  # faster and lighter than count() here
    .sort_values(ascending=False)
    .head(10)
)

# pd.DataFrame(top_arrivals, index=None).rename(columns={0: "Top Arrived and Registered"})

In [19]:
# Aggregate by arrived and count of full name
# final_df[['arrived', 'full_name']].groupby("arrived").count()

In [20]:
# TODO: Transpose the DataFrame by swapping names in rows and folder names in columns

# Step 2: Pivot the DataFrame
arrived_pivot = final_df.pivot_table(
    index='full_name',  # Use 'full_name' as the row index
    columns='folder_name',  # Use 'folder_name' as the columns
    values='arrived',  # Use 'arrived' as the values
    aggfunc='any'  # Aggregate using 'any' to ensure boolean values
)

# Step 3: Reset the index (optional)
arrived_pivot = arrived_pivot.reset_index()

# Step 4: Rename columns (optional)
arrived_pivot.columns.name = None  # Remove the column grouping name
arrived_pivot = arrived_pivot.rename_axis(None, axis=1)  # Remove the index name

# Replace NaN values with False
# arrived_pivot = arrived_pivot.fillna(False)

In [21]:
# arrived_pivot.to_csv("arrived_pivot.csv", index=False)

# duplicates = final_df[final_df["full_name"].duplicated(keep=False)]
# print(duplicates)

In [23]:
from gspread_dataframe import set_with_dataframe
# Load environment variables from the .env file
load_dotenv()

DRIVE_ROOT_PROJECT_FOLDER_ID = os.getenv("DRIVE_ROOT_PROJECT_FOLDER_ID")

def export_attendance_matrix_to_drive(pivot_df: pd.DataFrame, root_folder_id: str, gc, sheet_title="Oh Snap - Attendance Matrix"):

    metadata = {
        'name': sheet_title,
        'mimeType': 'application/vnd.google-apps.spreadsheet',
        'parents': [root_folder_id],
    }

    file = drive_service.files().create(body=metadata, fields='id').execute()
    spreadsheet_id = file['id']

    spreadsheet = gc.open_by_key(spreadsheet_id)
    worksheet = spreadsheet.sheet1

    set_with_dataframe(worksheet, pivot_df.reset_index())
    return f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}"

# export_attendance_matrix_to_drive(arrived_pivot, DRIVE_ROOT_PROJECT_FOLDER_ID, gc)